Introduction
============

This notebook was inspired by my experiences participating in Kaggle competitions. Since May of 2014 I had been participating in several [Kaggle competitions](http://www.kaggle.com). For the most part the competitions had consisted of 

1. There are 2 sources of categorical data (Cats and Dogs).
2. A competitor is supplied with labelled training data with which to generate models of the sources of data.
3. The competitor is supplied with unlabelled test data and is asked to then predict the correct label for each test data sample.
4. The competitor who has the most accurate predictions *wins* the contest.

After having participated in several contests I decided to get a better understanding of the problem of predicting labels of test data by making some of my own simulations and in the process I observed a pattern in the rate of success.


Estimating the Success Rate
===========================

I'm going to begin with a derivation of the success rate of prediction. 

* Suppose $X_1,X_2$ are categorical variables with the same set of possible values whose respective pdfs are $p_1(x),p_2(x)$. 
* Suppose $X$ is a categorical variable that is an even mixture of $X_1$ of $X_2$, that is, each time we sample from $X$ there is a $50\%$ chance we sample from $X_1$ and a $50\%$ chance we sample from $X_2$.

Now given a sequence of observed samples of $X$: $x_1,x_2, \ldots, x_N$ I want to answer several questions: 
* What is the probability that $x_i$ was sampled from $X_1$ (respectively $X_2$)?
* Which label should I predict if I observe $X=x$?
* What is my expected rate of success of accurately labelling $\{x_n\}_{n=1}^N$ for $N\gg 1$?

To answer the questions we begin with determining the probability distribution for $X$:
$$\begin{align}
Pr(X = x) &= Pr(X,\text{sampled from }X_1) + Pr(X,\text{sampled from }X_2) \\
&= Pr(X|\text{sampled from }X_1)Pr(\text{sampled from }X_1)\\
&+ Pr(X|\text{sampled from }X_2)Pr(\text{sampled from}X_2) \\
&= 0.5Pr(X|\text{sampled from }X_1) + 0.5Pr(X|\text{sampled from }X_2)\\
&= 0.5\bigl(p_1(x) + p_2(x)\bigr) 
\end{align}$$
In other words, the probability distribution is the average of the 2 probability distributions, $p_1(x), p_2(x)$.

##  What is the probability that $x$ was sampled from $X_1$ (respectively $X_2$)?
Now suppose $X = x$ is observed. What is the probability as to the source, $X_1$ or $X_2$? Using Bayes' theorem we can compute the probability that $X_1$ (respectively $X_2$) was the source.

$$\begin{align}
P(\text{sampled from }X_1| X=x) & = Pr(X=x| \text{sampled from }X_1)\frac{Pr(\text{samplef from }X_1)}{Pr(X=x)} \\
&= p_1(x)\frac{0.5}{0.5\bigl(p_1(x) +p_2(x)\bigr)}\\
P(\text{sampled from }X_1| X=x) &= \frac{p_1(x)}{p_1(x) + p_2(x)}
\end{align}$$
Similarly, $P(\text{sampled from }X_2| X=x) = p_2(x)/\bigl(p_1(x) + p_2(x)\bigr)$

Now I have an answer to the first question.

## Which label should I predict if I observe $X=x$?
For the second question, if I want to maximize the chances to correctly label an observed stream of data, the algorithm is simple. Whenever I observe $X=x$ I label the source as the one whose probability $p_1(x), p_2(x)$ is greatest.

## What is my expected rate of success of accurately labelling $\{x_i\}$ for $N\gg 1$?
For the third question, let $$f(x) := \max\{p_1(x),p_2(x)\},\quad  p_{max}(x) := \frac{f(x)}{p_1(x) + p_2(x)}$$

$p_{max}(x)$ is the rate of success in correctly labelling the source of $X=x$ if I always choose the source whose probability is greatest when $X=x$.

The expected rate of success is:

\begin{align}
E(p_{max}) &:= \sum_x Pr(X=x)p_{max}(x)\\
&= \sum_x 0.5(p_1(x)+p_2(x))p_{max}(x)\\
&= 0.5\sum_x f(x)
\end{align}

For large amounts of test data, $E(p_{max})$ is the limit of the accuracy rate in labelling the source of the data. It's the "speed of light" barrier that one can't overcome without additional information.

Monte Carlo Simulations
========================
Given this formula, I wanted to see what sort of success rates I could expect when the number of categorical values was $2^n$ for some positive integer $n$. The simplest thing I could think of was to perform some [Monte Carlo simulations](http://en.wikipedia.org/wiki/Monte_Carlo_method) and generate some statistics.

I will assume that $p_1(x), p_2(x)$ are sampled from the [Dirichlet distribution](http://en.wikipedia.org/wiki/Dirichlet_distribution) $Dirichlet((1,\ldots,1))$, i.e., each possible distribution on $2^n$ values is equally likely to be drawn.

In [ ]:
%pylab inline

In [ ]:
N=10 #  1 <= n <=10
M=10000 # Number of times to draw from the dirichlet distribution

expected_rates_of_succes = zeros((M,N+1))

In [ ]:
for n in range(1,N+1):
    categories = 2**n
    p1p2 = random.dirichlet(ones(categories),size=(2,M))
    f = p1p2.max(axis=0)
    expected_rates_of_succes[:,n] = 0.5*f.sum(axis=1)

In [ ]:
figure(figsize=(20,10))
for n in range(1,N+1):
    subplot(2,5,n)
    title("Success rate for $2^{%d}$"%n,fontsize=16)
    hist(expected_rates_of_succes[:,n],bins=100);
tight_layout()

As you can see the histograms of the expected success rates appear to be converging to a normal distribution. Let's do some statistics on the sample mean and standard deviation for the data.

In [ ]:
means = expected_rates_of_succes.mean(axis=0)
stds = expected_rates_of_succes.std(axis=0)

In [ ]:
figure(figsize=(16,6))
subplot(1,2,1)
title("sample mean for expected rate of success for $2^n$",fontsize=16)
xlabel("n")
plot(arange(1,N+1),means[1:]);
subplot(1,2,2)
title("sample standard deviation for expected rate of success for $2^n$",fontsize=16)
xlabel("n")
plot(stds[1:]);
tight_layout()

Let's see if we can't find curves to fit the mean and standard deviations.

Looking at the sample mean graph, it looks to me like the mean values are approaching $0.75$ at some exponential rate.

In [ ]:
offset_means = 0.75 - means
ylabel("log(mean expected success rate)")
xlabel("n")
plot(arange(1,11),log(offset_means[1:,]));

In [ ]:
import statsmodels.api as sm

In [ ]:
xs = arange(1,11)
ys = log(offset_means[1:])/log(2.0)
Xs = sm.add_constant(xs)

In [ ]:
the_data = sm.OLS(ys,Xs)
model = the_data.fit()
model.summary()

It looks like $\log_2\bigl(3/4 - \text{expected success rate}\bigr) \approx -2.65 - n$ or solving for the expected success rate, we get

$$\text{expected success rate} \approx 3/4 - 2^{-(n+5.3/2)}$$

What about the sample standard deviation? It doesn't look like exponential decay to me, maybe a reciprocal $y = 1/x$ model?

After some graphing experiments, I was able to find a linear relationship. The model I found was:

$$\text{sample std}_n \approx 2^{-5/2-n/2}$$

Here's the work:

In [ ]:
title("$2^n$ vs. $1/\sigma^2$",fontsize=16)
xlabel("$2^n$",fontsize=16)
ylabel("$1/\sigma^2$",fontsize=16)
plot(2**arange(1,11),(1/stds[1:])**2);

In [ ]:
std_xs = 2**arange(1,11)
std_ys = 1/stds[1:]**2
std_XS = sm.add_constant(std_xs)

In [ ]:
this_data = sm.OLS(std_ys,std_XS)
std_model = this_data.fit()
std_model.summary()

Based on the summary statistics, it looks like $1/\sigma^2 \approx 32\times 2^n = 2^{5+n}$, $\sigma^2 = 2^{-(5+n)}$, and solving for $\sigma$, $\sigma \approx 2^{-5/2 - n/2}$

The Type of Distribution for the Expected Success Rate
=============================================

Let's see what the [Kolmogorov-Smirnov Test](http://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) has to say about the expected success rates being sampled from a normal distribution.

In [ ]:
import scipy.stats

In [ ]:
for n in range(1,N+1):
    normalized_data = (expected_rates_of_succes[:,n] - means[n])/stds[n]
    print (n, "p value:",scipy.stats.kstest(normalized_data,"norm")[1])


So it looks like by the time we reach a random variable of $2^4 = 16$ categorical values, the expected success rate for correctly labelling the source of a value is close enough to declare it a Gaussian random variable.

Based on the simulations, it looks as if $\lim_{n\to \infty} \text{Expected Success Rate} = 0.75.$ Let's see if we can't derive the result mathematically.

Derivation of the Distribution Result
========================
I want to make a derivation of the result using some simplifications to make the computations easier to perform. For a Dirichlet distribution, the components are not independent, but I'm going to treat them as if they were. As a consequence since I'm summing a bunch of independent identically distributed variables, I should expect the mean to be normally distributed. That would explain the observed histograms. It's not that bad of an approximation for $n \gg 1$.

For a Dirichlet distribution $Dirichlet(\vec{\alpha})$, the mean and variances of the values of the components is given by 
$$ \mu_i = \frac{\alpha_i}{\sum_j \alpha_j}, \sigma^2_i = \frac{\alpha_i (\sum_{j} \alpha_j - \alpha_i)}{(\sum_j \alpha_j)^2(\sum_j \alpha_j - 1)}$$

while the covariance between between the $i$ and $j$ components is given by 

$$cov(i,j) = \frac{-\alpha_i\alpha_j}{(\sum_k \alpha_k)^2\bigl((\sum_k \alpha_k) +1\bigr)}$$

For $n \gg 1$,

\begin{align}
\mu_i &= 2^{-n}\\
\sigma^2_i &= \frac{2^n - 1}{2^{2n}(2^{n} - 1)}\\
&= 2^{-2n}\\
cov(i,j) &= \frac{-1}{2^{2n}(2^n + 1)}\\
&\approx -2^{-3n} \approx  0
\end{align}
since $\alpha_i \equiv 1$. 


Each component of the Dirichlet distribution $Dir(1,\ldots, 1)$ can be modelled as a [Beta distribution](http://en.wikipedia.org/wiki/Beta_distribution) $Beta(1,2^n -1)$ with the pdf given by $p(x) = (2^n -1)(1-x)^{2^n - 2}$

For 2 independent identically distributed $Beta(1,2^n - 1)$ variables $X_1,X_2$ 

\begin{align}
Pr(max(X_1,X_2) = c) &= 2p(c)\int_0^c p(x)dx \\
&= 2p(c)[-(1-x)^{2^n-1}]_0^c \\
&= 2p(c)\bigl(1 - (1-c)^{2^n-1}\bigr) \\
&= 2p(c) - 2(2^n-1)(1-c)^{2^n-2}(1-c)^{2^n-1} \\
&= 2p(c) - (2^{n+1} -2)(1-c)^{2^{n+1}-3}
\end{align}

This distribution is a linear combination of 2 Beta distributions, $Beta(1,2^n -1)$ and $Beta(1,2^{n+1} - 2)$. The mean and variance are given by 

\begin{align} 
\mu &= 2\times 2^{-n} - \frac{1}{2^{n+1} - 1}, & \sigma^2 = 4\times\frac{2^n -1}{(2^n)^2(2^n + 1)} + \frac{2^{n+1} -2}{(2^{n+1} - 1)^2(2^{n+1})} \\
&= 2^{-n+1} - \frac{1}{2^{n+1} - 1}, &=  \frac{2^{n+2} - 4}{2^{2n}(2^n + 1)} + \frac{2^{n+1} -2 }{(2^{n+1} - 1)^2(2^{n+1})}\\
&\approx 2^{-n+1} - 2^{-n-1}, n\gg 1, &\approx 2^{-2n + 2} + 2^{-2n}, n \gg 1 \\
&=2^{-n+1}(1 - 2^{-2}), n \gg 1, &= 2^{-2n}(2^2 + 1), n \gg 1\\
&=(3/4) 2^{-n+1}, n\gg 1, &= (5)2^{-2n}, n\gg 1
\end{align}

To compute the expected success rate, we sum over all the $2^n$ *independent* components and then multiply by 0.5.

\begin{align}
\text{expected success rate} &\approx  0.5 \times 2^n (3/4)2^{-n+1} \\
&= 1/2 \times (3/4) 2^{n-n+1}\\
&= 1/2 \times 3/4 \times 2 = 3/4.
\end{align}

Or to be more precise,

\begin{align}
\text{expected success rate} &\approx 0.5 \times 2^{n}\bigl(2^{-n+1} - \frac1{2^{n+1} - 1}\bigr) \\
&= 1 - \frac{2^{n-1}}{2^{n+1} - 1} \\
&= 1 - \frac{2^{-2}}{1 - 2^{-(n+1)}}\\
&\approx 1 - 2^{-2}\bigl(1 + 2^{-(n+1)}\bigr)\\
&= 1 - 2^{-2} - 2^{-(n+3)}\\
&= 3/4 - 2^{-(n+3)} 
\end{align}

The variance in the expected success rate is similarly approximated to be:

\begin{align}\text{variance in expected success rate} &\approx 0.5^2 \times 2^{n}(5)(2^{-2n}) \\
& = (5) 2^{-2 + n - 2n}\\
&= (5)2^{-(n+2)}\\
&\approx 2^{-n+.32}
\end{align}

So while these formulas don't match exactly the estimated formulas from sample data they do give evidence to support the limit of the expected accuracy rate to be $3/4$ and that the rate of decrease in variance is $O(2^{-n})$.

The Effect of Independent Variables
===================================

I want to understand the effect of a distribution that's a product of distributions on the expected success rate. I'm going to do some more Monte Carlo simulations to build an intuition.

In [ ]:
N=10 #  1 <= n <=10
M=10000 # Number of times to draw from the dirichlet distribution

expected_rates_of_succes = zeros((M,N+1,N+1))

In [ ]:
for n1 in range(1,N):
    categories1 = 2**n1
    for n2 in range(n1,N+1):
        categories2 = 2**n2
        for m in range(M):
            stack_space = zeros((categories1*categories2,2))
            p1p2_1 = random.dirichlet(ones(categories1),size=2)
            p1p2_2 = random.dirichlet(ones(categories2),size=2)
        
            cross_1 = (p1p2_1[0]).reshape((categories1,1))*(p1p2_2[0]).reshape((1,categories2))
            cross_2 = (p1p2_1[1]).reshape((categories1,1))*(p1p2_2[1]).reshape((1,categories2))
            cross_1 = cross_1.reshape(categories1*categories2)
            cross_2 = cross_2.reshape(categories1*categories2)
            stack_space[:,0] = cross_1
            stack_space[:,1] = cross_2
            f = stack_space.max(axis=1)
            expected_rates_of_succes[m,n1,n2] = 0.5*f.sum()

In [ ]:
t = 1
figure(figsize=(24,30))
for i in range(1,N):
    for j in range(i,N+1):
        subplot(9,6,t)
        title("Product of $2^{%d} * 2^{%d}$" % (i,j),fontsize=16)
        hist(expected_rates_of_succes[:,i,j],bins=100);
        t = t +1
tight_layout()

Let's take a look at the summary statistics:

In [ ]:
print("Sample mean and std")
for i in range(1,N):
    for j in range(i,N+1):
        mu = expected_rates_of_succes[:,i,j].mean()
        sigma = expected_rates_of_succes[:,i,j].std()
        print("for 2**%d x 2**%d: %8.6f %8.6f" % (i,j,mu,sigma))

So it looks like a product of 2 distributions has a larger expected success rate. By an induction argument I'm going to guess that as the number of factors that go into a product distribution increases, the expected success rate will also increase.

Suddenly it becomes clear to me why the [naive Bayes classifier](http://en.wikipedia.org/wiki/Naive_Bayes_classifier) method is so popular and effective --- if the joint distribution can be teased apart into a product of distributions, the expected success rate goes up from a single distribution baseline.

# Example: Titanic data set
I want to try to predict from the titanic data set if a passenger was male vs. female. What I would like to determine is the maximum rate of success I could expect.

**NB**: you'll need a local copy of the Kaggle titanic data set for this section of the notebook to work.

My 2 competing probability distributions:

* $p_1(x) := Pr(\text{survived yes/no, passenger class} | female)$
* $p_2(x) := Pr(\text{survived yes/no, passenger class} | male)$

Objective: predict male/female given the survival status and passenger class.

In [ ]:
import pandas as pd
df = pd.read_csv("kaggle/titanic/train.csv")

In [ ]:
df.head()

In [ ]:
df.groupby("Sex").count()

So my data set is not evenly divided. How does this affect my predictive capabilities?

$$f_{max}(x) := \max\{\alpha p_1(x), (1-\alpha) p_2(x)\}, \quad p_{max}(x) = \frac{f_{max}(x)}{\alpha p_1(x) + (1-\alpha)p_2(x)}$$

In [ ]:
alpha = df.groupby("Sex").count().loc['female']['Survived']/df.shape[0]
dg = df.groupby(["Sex",'Survived','Pclass'])
dh = dg.count()
the_counts = dh.PassengerId
my_keys = the_counts.keys()

In [ ]:
counts1 = zeros(6,dtype=int).reshape(2,3)
counts2 = zeros(6,dtype=int).reshape(2,3)

for akey in my_keys:
    survived, my_class = akey[1:]
    if akey[0] == 'female':
        counts1[survived,my_class-1] = the_counts[akey]
    else:
        counts2[survived,my_class-1] = the_counts[akey]

In [ ]:
counts1 += 1
counts2 += 1
counts1 = counts1.flatten()
counts2 = counts2.flatten()

In [ ]:
print(counts1)
print(counts2)

In [ ]:
def sample_dirichlet(counts,T=1000):
    """Sample T distribution from the Dirichlet distribution Dir(counts)"""
    samples = np.random.dirichlet(counts,size=T)
    return samples

def compute_fmax(p1,p2):
    """Takes 2 distributions and compute the max of each entry:"""
    temp = np.hstack([p1.reshape(-1,1),p2.reshape(-1,1)])
    return temp.max(axis=-1)

def monte_carlo(counts1, counts2, alpha, T=1000):
    """Performs T samples from the 2 counts Dirichlet
    distributions, compute the sample fmax, and returns the results."""
    
    p1 = sample_dirichlet(counts1,T=T)
    p2 = sample_dirichlet(counts2,T=T)
    f_max = array([compute_fmax(alpha*p1[t],(1-alpha)*p2[t]) for t in range(T)])
    scores = f_max.sum(axis=1)
    return scores.mean(),scores.std()

In [ ]:
mean_success, std_success = monte_carlo(counts1,counts2,alpha)
print("Mean rate of successful prediction: ", mean_success)
print("Std rate of successful prediction: ", std_success)